In [36]:
# File renaming for consistency
import os
from os import path

for count, filename in enumerate(os.listdir("deadlift2")):
    src = "deadlift2/" + filename
    string = "deadlift2/deadlift_" + str(count) + ".jpg"
    os.rename(src, string)

https://machinelearningmastery.com/how-to-develop-a-convolutional-neural-network-to-classify-photos-of-dogs-and-cats/

In [37]:
from os import listdir
from numpy import asarray
from numpy import save
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array

folder = "train/"
photos, labels = list(), list()

for file in listdir(folder):
    output = 0.0
    if file.startswith("squat"):
        output = 1.0
    if file.startswith("deadlift"):
        output = 2.0
    photo = load_img(folder + file, target_size=(150,150))
    photo = img_to_array
    
    photos.append(photo)
    labels.append(output)
photos = asarray(photos)
labels = asarray(output)
print(photos.shape, labels.shape)

save("exercise_photos.npy", photos)
save("exercise_labels.npy", photos)

(549,) ()


In [39]:
from numpy import load
photos = load("exercise_photos.npy",allow_pickle=True)
labels = load("exercise_labels.npy",allow_pickle=True)

print(photos.shape, labels.shape)

(549,) (549,)


In [42]:
# Directory Generation
from os import makedirs
dataset_home = "dataset/"
subdirs = ["train/", "test/"]
for subdir in subdirs:
    # create label subdirectories
    labeldirs = ["bench/", "squat/", "deadlift/"]
    for labldir in labeldirs:
        newdir = dataset_home + subdir + labldir
        makedirs(newdir)

In [46]:
# Segment into testing and training images
import random
from shutil import copyfile
random.seed(1)
ratio = 0.2
dataset_home = "dataset/"
src_directory = "images/"
for file in listdir(src_directory):
    src = src_directory + '/' + file
    dst_dir = "train/"
    if random.random() < ratio:
        dst_dir = "test/"
    if file.startswith("bench"):
        dst = dataset_home + dst_dir + "bench/" + file
    elif file.startswith("squat"):
        dst = dataset_home + dst_dir + "squat/" + file
    else:
        dst = dataset_home + dst_dir + "deadlift/" + file
    copyfile(src, dst)   

In [70]:
# Baseline CNN Model
import sys
from matplotlib import pyplot
import keras
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

 
# one block VGG
"""
def define_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(150, 150, 3)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(1, activation='sigmoid'))
    # compile model
    opt = SGD(lr=0.001, momentum=0.9)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return model
"""
"""
# two block VGG
def define_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(150, 150, 3)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(1, activation='sigmoid'))
    # compile model
    opt = SGD(lr=0.001, momentum=0.9)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return model
"""


def define_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1, activation='softmax'))
    # compile model
    #opt = SGD(lr=0.001, momentum=0.9)
    model.compile(optimizer=keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

"""
# three block VGG
def define_model():

    cnn1 = Sequential()
    cnn1.add(Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
    cnn1.add(MaxPooling2D((2, 2)))
    cnn1.add(Dropout(0.2))

    cnn1.add(Flatten())

    cnn1.add(Dense(128, activation='relu'))
    cnn1.add(Dense(10, activation='softmax'))

    cnn1.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])
    return cnn1

"""

# plot diagnostic learning curves
def summarize_diagnostics(history):
    # plot loss
    pyplot.subplot(211)
    pyplot.title('Cross Entropy Loss')
    pyplot.plot(history.history['loss'], color='blue', label='train')
    pyplot.plot(history.history['val_loss'], color='orange', label='test')
    # plot accuracy
    pyplot.subplot(212)
    pyplot.title('Classification Accuracy')
    pyplot.plot(history.history['accuracy'], color='blue', label='train')
    pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
    # save plot to file
    filename = sys.argv[0].split('/')[-1]
    pyplot.savefig(filename + '_plot.png')
    pyplot.close()
 
# run the test harness for evaluating a model
def run_test_harness():
    # define model
    model = define_model()
    # create data generator
    datagen = ImageDataGenerator(rescale=1.0/255.0)
    # prepare iterators
    train_it = datagen.flow_from_directory('dataset/train/',
        class_mode='categorical', batch_size=128, target_size=(150, 150))
    test_it = datagen.flow_from_directory('dataset/test/',
        class_mode='categorical', batch_size=128, target_size=(150, 150))
    # fit model
    history = model.fit(train_it, steps_per_epoch=len(train_it),
        validation_data=test_it, validation_steps=len(test_it), epochs=20, verbose=0)
    # evaluate model
    _, acc = model.evaluate_generator(test_it, steps=len(test_it), verbose=0)
    print('> %.3f' % (acc * 100.0))
    # learning curves
    summarize_diagnostics(history)
 
# entry point, run the test harness
run_test_harness()


Found 448 images belonging to 3 classes.
1
Found 101 images belonging to 3 classes.


InvalidArgumentError:  Matrix size-incompatible: In[0]: [128,3], In[1]: [128,1]
	 [[node gradient_tape/sequential_21/dense_41/MatMul (defined at <ipython-input-70-ca63fab2532d>:115) ]] [Op:__inference_train_function_17586]

Function call stack:
train_function
